In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.contrib import lite

In [3]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [6]:
#importing keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import regularizers
from keras.models import Model

In [7]:
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary()
type(vgg16_model)
vgg16_model.layers.pop()
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
model = Sequential()

for layer in vgg16_model.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False


model.add(Dense(10,activation= 'softmax'))

inputs=vgg16_model.input
outputs=model.output

model=Model(inputs,outputs)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
# model.summary()
# Compiling the CNN
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])
#model.optimizer.lr = 0.001
# Fitting the CNN to the images
# Image augmentation for reducing overfitting - Balancing bias and variance



from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('Monkey/training',
                                                    target_size=(224, 224),
                                                    batch_size= 32,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory('Monkey/validation',
                                                        target_size=(224,224),
                                                        batch_size=16,
                                                        class_mode='categorical')

train_generator.image_shape
#steps_per_epoch - no of training data image
#validation_steps - no of testing data image
model.fit_generator(train_generator,
                         steps_per_epoch=34,       #1097/32
                         epochs=50,
                         validation_data=validation_generator,
                         validation_steps=17)   #272/16

Found 1097 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
Epoch 1/1
34/34 [==============================] - 34s 1s/step - loss: 2.2896 - acc: 0.2160 - val_loss: 1.9071 - val_acc: 0.3419


In [10]:
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
#                                     # (nothing gets printed in Jupyter, only if you run it standalone
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

# output_names = [node.op.name for node in model.outputs]
# # sess = tf.keras.backend.get_session()
# sess.run(tf.global_variables_initializer())
# frozen_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_names)

# print("tflite model starts..")
# # # Converting a GraphDef from file.
# with tf.device("/gpu:0"):
#     converter = lite.TFLiteConverter.from_frozen_graph(frozen_def,inputs,output_names)
# #     converter.inference_type = tf.lite.constants.QUANTIZED_UINT8

# print("tflite model conversion taking place...")
# tflite_model = converter.convert()
# print("tflite model writing...")
# open("converted_model.tflite", "wb").write(tflite_model)
# print("tflite model ends !!")
# print(output_names)
# print(inputs)

In [ ]:
sess = tf.keras.backend.get_session()
sess.run(tf.global_variables_initializer())

output_names = [node.op.name for node in model.outputs]
frozen_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_names)

# Convert to TensorFlow Lite model.
print("tflite model starts..")
converter = converter = lite.TFLiteConverter.from_frozen_graph(frozen_def,inputs,output_names)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)
print("tflite model ends..")

INFO:tensorflow:Froze 32 variables.
INFO:tensorflow:Converted 32 variables to const ops.
tflite model starts..


In [ ]:
# # Converting a GraphDef from session.
# converter = lite.TFLiteConverter.from_session(sess, in_tensors, out_tensors)
# tflite_model = converter.convert()
# open("converted_model.tflite", "wb").write(tflite_model)

# # Converting a GraphDef from file.
# converter = lite.TFLiteConverter.from_frozen_graph(
#   graph_def_file, input_arrays, output_arrays)
# tflite_model = converter.convert()
# open("converted_model.tflite", "wb").write(tflite_model)

# # Converting a SavedModel.
# converter = lite.TFLiteConverter.from_saved_model(saved_model_dir)
# tflite_model = converter.convert()

# # Converting a tf.keras model.
# converter = lite.TFLiteConverter.from_keras_model_file(keras_model)
# tflite_model = converter.convert()